## Импортирование необходимых библиотек

In [104]:
import glob
import os
from tqdm import tqdm
import pandas as pd
import random
import dspy
import warnings
warnings.filterwarnings('ignore')
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

## Подключение к локальной лламе

In [ ]:
my_llama = dspy.OllamaLocal(model = "llama3.1:8b", model_type = "text", temperature = 0.0, max_tokens = 1000, top_p = 0.8, top_k = 40)
gpt4mini = dspy.LM("openai/gpt-4o-mini",
             api_base = "https://api.proxyapi.ru/openai/v1",
             api_key = "")
dspy.settings.configure(lm = gpt4mini)

In [106]:
RUN_FROM_SCRATCH = False

## Загрузка данных

In [107]:
data = pd.read_csv("Toxic_Russian_Comments.csv")
# data['toxic'] = data['toxic'].apply(lambda x: 'Toxic' if x == 1.0 else 'Not Toxic')

def as_example():
    dataset = []
    for i in range(1, 1000):
        statement = data.iloc[i]["comment"]
        question = f"""Оцени, является ли это высказывание токсичным: "{statement}". Отвечай только 0 или 1, где 0 означает, что в высказывании нет токсичности, а 1 - что есть"""
        answer = str(int(data.iloc[i]["toxic"]))
        example = dspy.Example({
            "question": question,
            "answer": answer}).with_inputs("question")
        dataset.append(example)
    return dataset
dataset = as_example()
print(dataset)

[Example({'question': 'Оцени, является ли это высказывание токсичным: "Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n". Отвечай только 0 или 1, где 0 означает, что в высказывании нет токсичности, а 1 - что есть', 'answer': '1'}) (input_keys={'question'}), Example({'question': 'Оцени, является ли это высказывание токсичным: "Собаке - собачья смерть\n". Отвечай только 0 или 1, где 0 означает, что в высказывании нет токсичности, а 1 - что есть', 'answer': '1'}) (input_keys={'question'}), Example({'question': 'Оцени, является ли это высказывание токсичным: "Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n". Отвечай только 0 или 1, где 0 означает, что в высказывании нет токсичности, а 1 - что есть', 'answer': '1'}) (input_keys={'question'}), Example({'question': 'Оцени, является ли

### Train and dev samples

In [108]:
train, dev = dataset[: 20], dataset[400: 800]
len(train), len(dev)

(20, 400)

## Zero-shot CoT

In [ ]:
# def compute_accuracy():
#     predv = dspy.ChainOfThought(Toxicity)
#     CoTs = []
#     results = []
#     for i in tqdm(range(20)):
#         CoT = predv(question = dataset[i]["question"])
#         CoTs.append(CoT)
#         if int(CoT.answer) == int(data.iloc[i]["toxic"]):
#             results.append(1)
#         else:
#             results.append(0)
#     accuracy = sum(results) / len(results) * 100
#     return accuracy
# accuracy = compute_accuracy

class Toxicity(dspy.Signature):
    ("""Тебе дано утверждение, которое потенциально может быть токсичным.
    Ты должен оценить, является ли токсичным данное тебе утверждение и вывести в ответ только 1 или 0. Единица означает наличие токсичности, ноль - ее отсутствие в тексте. Ты должен рассуждать только на русском языке""")
    question = dspy.InputField()
    answer = dspy.OutputField(desc = "1 или 0")


class ToxicCoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_answer = dspy.ChainOfThought(Toxicity)

    def forward(self, question):
        return self.generate_answer(question = question)

CoT = ToxicCoT()
toxicity_accuracy = dspy.evaluate.metrics.answer_exact_match
evaluator = Evaluate(devset = train, num_threads = 1, display_progress = True, display_table = 0, provide_traceback = True)
evaluator(CoT, metric = toxicity_accuracy)


  0%|          | 0/20 [00:00<?, ?it/s]

2024/11/18 17:45:17 ERROR dspy.evaluate.evaluate: Error for example in dev set: 		 compute_accuracy() takes 0 positional arguments but 2 were given
	with inputs:
		Example({'question': 'Оцени, является ли это высказывание токсичным: "Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n". Отвечай только 0 или 1, где 0 означает, что в высказывании нет токсичности, а 1 - что есть'}) (input_keys={'question'})

Stack trace:
	Traceback (most recent call last):
  File "c:\Users\Юлия\Desktop\petprojects\dspy\env_for_dspy\Lib\site-packages\dspy\evaluate\evaluate.py", line 179, in wrapped_program
    score = metric(
            ^^^^^^^
TypeError: compute_accuracy() takes 0 positional arguments but 2 were given



Average Metric: 0.0 / 1  (0.0):   0%|          | 0/20 [00:00<?, ?it/s]

2024/11/18 17:45:17 ERROR dspy.evaluate.evaluate: Error for example in dev set: 		 compute_accuracy() takes 0 positional arguments but 2 were given
	with inputs:
		Example({'question': 'Оцени, является ли это высказывание токсичным: "Собаке - собачья смерть\n". Отвечай только 0 или 1, где 0 означает, что в высказывании нет токсичности, а 1 - что есть'}) (input_keys={'question'})

Stack trace:
	Traceback (most recent call last):
  File "c:\Users\Юлия\Desktop\petprojects\dspy\env_for_dspy\Lib\site-packages\dspy\evaluate\evaluate.py", line 179, in wrapped_program
    score = metric(
            ^^^^^^^
TypeError: compute_accuracy() takes 0 positional arguments but 2 were given



Average Metric: 0.0 / 2  (0.0):   5%|▌         | 1/20 [00:00<00:00, 333.25it/s]

2024/11/18 17:45:17 ERROR dspy.evaluate.evaluate: Error for example in dev set: 		 compute_accuracy() takes 0 positional arguments but 2 were given
	with inputs:
		Example({'question': 'Оцени, является ли это высказывание токсичным: "Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n". Отвечай только 0 или 1, где 0 означает, что в высказывании нет токсичности, а 1 - что есть'}) (input_keys={'question'})

Stack trace:
	Traceback (most recent call last):
  File "c:\Users\Юлия\Desktop\petprojects\dspy\env_for_dspy\Lib\site-packages\dspy\evaluate\evaluate.py", line 179, in wrapped_program
    score = metric(
            ^^^^^^^
TypeError: compute_accuracy() takes 0 positional arguments but 2 were given



Average Metric: 0.0 / 3  (0.0):  10%|█         | 2/20 [00:00<00:00, 396.27it/s]

2024/11/18 17:45:17 ERROR dspy.evaluate.evaluate: Error for example in dev set: 		 compute_accuracy() takes 0 positional arguments but 2 were given
	with inputs:
		Example({'question': 'Оцени, является ли это высказывание токсичным: "тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n". Отвечай только 0 или 1, где 0 означает, что в высказывании нет токсичности, а 1 - что есть'}) (input_keys={'question'})

Stack trace:
	Traceback (most recent call last):
  File "c:\Users\Юлия\Desktop\petprojects\dspy\env_for_dspy\Lib\site-packages\dspy\evaluate\evaluate.py", line 179, in wrapped_program
    score = metric(
            ^^^^^^^
TypeError: compute_accuracy() takes 0 positional arguments but 2 were given



Average Metric: 0.0 / 4  (0.0):  15%|█▌        | 3/20 [00:00<00:00, 425.76it/s]

TypeError: compute_accuracy() takes 0 positional arguments but 2 were given

## Optimized few-shot with bootstrapped demonstrations

In [111]:
bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
    max_bootstrapped_demos = 8,
    max_labeled_demos = 8,
    num_candidate_programs = 10,
    num_threads = 8,
    metric = toxicity_accuracy,
    teacher_settings = dict(lm = gpt4mini))

Going to sample between 1 and 8 traces per predictor.
Will attempt to bootstrap 10 candidate sets.


In [112]:
cot_fewshot = bootstrap_optimizer.compile(cot_zeroshot, trainset = train, valset = dev)


NameError: name 'cot_zeroshot' is not defined

In [ ]:
# evaluator(cot_fewshot, metric=scone_accuracy)

Average Metric: 141 / 200  (70.5): 100%|██████████| 200/200 [04:44<00:00,  1.42s/it]

2024/11/09 23:47:36 INFO dspy.evaluate.evaluate: Average Metric: 141 / 200 (70.5%)


70.5

In [ ]:
cot_fewshot.save("scone-cot_fewshot-turbo-gpt4-demos.json")